vamos a morirnos

In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from pymongo import MongoClient
from pandas import json_normalize
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry
import geopandas as gdp
from bson.json_util import dumps

In [2]:
df = pd.read_csv('newyork_data.csv', index_col = 0)
df2 = pd.read_csv('mad_data.csv', index_col = 0)
df3 = pd.read_csv('bcn_data.csv', index_col = 0)

In [3]:
def funcionquefunciona (df,ciudad):
    '''
    Esta función va a recibir un dataframe y una ciudad y lo que va a hacer es, transformar ese dataframe en un
    geodataframe generando las coordenadas como las lee mongo db y creando una nueva columna con las mismas (por cada
    uno de los estableciemientos en nuestro dataframe original). Seguidamente, se conecta con Mongo db y nos crea 
    la base de datos con sus coleccions (la ciudad que le pasamos) y genera un index de tipo 2dspehere para cada valor
    de la nueva columna de coordenadas generada antes, para poder hacer posteriormente las geoqueries.
    Recibe: un dataframe y una ciudad
    Devuelve: un mensaje que nos indica que la base de datos con las colecciones y los index han sido creados con éxito
    
    '''
    gdf = gdp.GeoDataFrame(df, geometry= gdp.points_from_xy(df.lng, df.lat ))
    print(gdf.columns)
    gdf.columns=['nombre', 'lat','lng','ciudad','distancia','loc','loc2' ]
    
    gdf['loc2']= gdf['loc2'].apply(lambda x:shapely.geometry.mapping(x))
    
    client = MongoClient()
    db = client.GeoProject
    collection = db.create_collection(name = f'{ciudad}')
    collection = db[f'{ciudad}']
    collection.create_index([("loc2", "2dsphere")])
    data = gdf.to_dict(orient='records')
    collection.insert_many(data)
    
    return 'Buenisima'

In [4]:
funcionquefunciona(df2,'mad')

'Buenisima'

In [5]:
funcionquefunciona(df3,'bnc')

'Buenisima'

In [20]:
lista1 = [df,df2,df3]
lista2 = ['ny', 'mad','bcn']

In [21]:
for i,j in zip(lista1,lista2):
    funcionquefunciona(i,j)

Index(['nombre', 'lat', 'lng', 'ciudad', 'distancia', 'loc', 'geometry'], dtype='object')
Index(['nombre', 'lat', 'lng', 'ciudad', 'distancia', 'loc', 'geometry'], dtype='object')
Index(['nombre', 'lat', 'lng', 'ciudad', 'distancia', 'loc', 'geometry'], dtype='object')


#### Pasamos a hacer las geoqueries

Primero nos tenemos que conectar con la base de datos en Mongo

In [2]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client.get_database("GeoProject")

In [3]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'GeoProject')

Guardamos en variables las llamadas a las colecciones de Mongo para un uso mas cómodo

In [4]:
m = db.get_collection("mad")
b = db.get_collection("bcn")
ny = db.get_collection('ny')

Empezamos con las geoqueries

In [57]:
# estas son las coordenadas que utilizaremos y en el formato que se tienen que usar (long, lat)
coord_ny = [-73.98996, 40.75673]
coord_mad = [2.19763, 41.40080]
coord_bcn = [-3.67097, 40.50222]

In [8]:
query = [{'$geoNear': {'near':[-73.98996, 40.75673],
                      'distanceField' : 'distance',
                       'maxDistance': 6000,
                       'distanceMultiplier': 6371,
                       'spherical': True}}]

In [9]:
geoloc_ny = ny.aggregate(query)
ny_response_json = json.loads(dumps(geoloc_ny))

In [10]:
query2 =[{'$geoNear': {'near':[2.19763, 41.40080],
                      'distanceField' : 'distance',
                       'maxDistance': 6000,
                       'distanceMultiplier': 6371,
                       'spherical': True}}]

In [11]:
geoloc_mad = m.aggregate(query2)
mad_response_json = json.loads(dumps(geoloc_mad))

In [87]:
coord = [[-73.98996, 40.75673], [2.19763, 41.40080], [-3.67097, 40.50222]]
colecciones = [ny, m , b]

In [5]:
# no funciona bien, me dice que en aggregate un string no tiene esa cualidad, pero le estoy pasando una coleccion ... ?
def geoquery(coordenadas, coleccion): #las coordenadas en formato lista (long y lat)
    '''
    Esta función, a partir de unas coordenadas y de una colección que tenemos creada en Mongo db, nos devuelve un json. 
    Esto lo hace a través de la query geoNear, donde esecificamos los criterios que debe seguir la misma. Después 
    llamamos a la colección y agregamos la query a la misma y a la respuesta que nos da le aplicamos un dumps y nos 
    devuelve un json con los ruquerimientos especificados.
    Recibe: coordenadas (formato lista) y colección de Mongo db (previamente hecha la conexión)
    Devuelve: json
    '''
    query = [{'$geoNear': {'near':f'{coordenadas}',
                      'distanceField' : 'distance',
                       'maxDistance': 6000,
                       'distanceMultiplier': 6371,
                       'spherical': True}}]
    
    col = f'{coleccion}'
    geoloc = col.aggregate(query)
    response_json = json.loads(dumps(geoloc))
    
    return response_json

In [12]:
query3 =[{'$geoNear': {'near':[-3.67097, 40.50222],
                      'distanceField' : 'distance',
                       'maxDistance': 6000,
                       'distanceMultiplier': 6371,
                       'spherical': True}}]

In [13]:
geoloc_bcn = b.aggregate(query3)
bcn_response_json = json.loads(dumps(geoloc_bcn))

Las distancias de 'distance' no tienen sentido en Madrid y Barcelona, en Nueva York coinciden y tienen snetido (comprobado), sin embargo tanto en Madrid como en Barcelona no (comprobado también, respecto a las coordenadas impuestas por mí). 
Por ello pasamos a utilizar la columna de distancia que se generó directamente en el primer json en vez de la de distance que hemos generado con el geoNear.

(Es mentira  que desde donde he puesto las coordenadas de Madrid, hasta el resturante vegano 'La huella Vegana' hay 500m, yo vivo al lado de este restaurante y hay bastnates kilómetros desde Las Rozas hasta la zona de 'La Vela' del BBVA).

In [24]:
def toDataFrame (df):
    '''
    Esta función recibe un dataframe y nos lo limpia y filtra por la distancia que le especifiquemos.
    Recibe: dataframe
    Devuelve: dataframe filtrado 
    '''
    
    df = df[['nombre', 'ciudad', 'distancia']]
    df = df[df['distancia'] < 6000]
    return df

In [39]:
ny = toDataFrame(json_normalize(ny_response_json))

In [33]:
mad = toDataFrame(json_normalize(mad_response_json))

In [35]:
bcn = toDataFrame(json_normalize(bcn_response_json))

In [36]:
ny.head()

,nombre,ciudad,distancia
0,Starbucks,New York,55
1,CBOA Pick Up/Drop Off Point #1,New York,69
2,Starbucks,New York,83
3,Applebee's Grill + Bar,New York,96
4,TerraCycle: Green Up Shop,New York,144


In [37]:
mad.head()

,nombre,ciudad,distancia
9,"Edificio Veganova, Alcobendas",Madrid,5476
10,Bar y Restaurante Payger,Alcobendas,5676
11,Bar Sube y Baja,Alcobendas,5293
13,Starbucks Diversia Alcobendas,Alcobendas,4172
15,Eventos Fly Up!,Madrid,4731


In [38]:
bcn.head()

,nombre,ciudad,distancia
18,Starbucks Diagonal 593,Barcelona,5753
19,Starbucks On The Go,Barcelona,5523
20,Secrid Pop Up Store,Barcelona,5443
21,What's Up! Berlín,Barcelona,5409
22,Starbucks Diagonal 640,Barcelona,5225


Ahora ya tenemos en 3 dataframes diferentes cada una de las ciudades con  nuestras 5 prioridades, no más lejos de 6 kilómetros.

Lo que tenemos que hacer ahora, es agrupar cada una de estas prioridades y sacar la media dela distancia de las mismas, seguidamente se ponderará cada una por el valor escogido según la prioridad que tenga frente a las otras. Tendremos un valor por cada una de las 5 prioridades estudiadas y por cada ciudad, por ello tendremos que realizar otra operación para quedarnos solo con un número por ciudad, el cual será el indicativo de cuan buena es esa ciudad según las características que hemo impuesto para la apertura de la empresa.